In [ ]:
# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. 
# This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

In [ ]:
# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. 
# All changes under this directory will be kept even after reset. 
# Please clean unnecessary files in time to speed up environment loading. 
!ls /home/aistudio/work

In [ ]:
# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, 
# you need to use the persistence path as the following: 
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

In [ ]:
!ls /home/aistudio/data/data154549/train_data_01/train_data_01/
import paddle
print(paddle.device.get_device())

# 数据准备
1.将提供的10个数据集合并成一个数据集

import glob
import shutil
import os

inpath = r"D:\BaiduNetdiskDownload\train_data_01\train_data_01\blur_image"
tarpath = r"D:\BaiduNetdiskDownload\train_data_01\train_data_01\gt_image" 

for i in range(2,10):
    print(os.path.join(r"D:\BaiduNetdiskDownload\train_data_0"+str(i),"train_data_0"+str(i)+r"\blur_image\*.png"))
    input1 = glob.glob(os.path.join(r"D:\BaiduNetdiskDownload\train_data_0"+str(i),"train_data_0"+str(i)+r"\blur_image\*.png"))
    target = glob.glob(os.path.join(r"D:\BaiduNetdiskDownload\train_data_0"+str(i),"train_data_0"+str(i)+r"\gt_image\*.png"))    
    print(input1)
    for j,k in zip(input1,target):
        shutil.move(j,os.path.join(inpath,os.path.basename(j)))
        shutil.move(k,os.path.join(tarpath,os.path.basename(k)))
               
for i in [10]:
    print(os.path.join(r"D:\BaiduNetdiskDownload\train_data_0"+str(i),"train_data_0"+str(i)+r"\blur_image\*.png"))
    input1 = glob.glob(os.path.join(r"D:\BaiduNetdiskDownload\train_data_"+str(i),"train_data_"+str(i)+r"\blur_image\*.png"))
    target = glob.glob(os.path.join(r"D:\BaiduNetdiskDownload\train_data_"+str(i),"train_data_"+str(i)+r"\gt_image\*.png"))    
    print(input1)
    for j,k in zip(input1,target):
        shutil.move(j,os.path.join(inpath,os.path.basename(j)))
        shutil.move(k,os.path.join(tarpath,os.path.basename(k)))


**1.构造数据读取器**

In [1]:
import paddle

import os
import numpy as np
import pandas as pd
import random

import cv2

class MyDateset(paddle.io.Dataset):
    def __init__(self, mode = 'train', watermark_dir = 'data/data154549/train_data_01/train_data_01/blur_image/', bg_dir = 'data/data154549/train_data_01/train_data_01/gt_image/'):
        super(MyDateset, self).__init__()

        self.mode = mode 
        self.watermark_dir = watermark_dir
        self.bg_dir = bg_dir

        self.train_list = os.listdir(self.watermark_dir)

    def __getitem__(self, index):
        item = self.train_list[index]
        #bg_item = item[:14]+'.jpg'

        img = cv2.imread(self.watermark_dir+item)
        label = cv2.imread(self.bg_dir+item)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        label = cv2.cvtColor(label, cv2.COLOR_BGR2RGB)
        #img = paddle.vision.transforms.resize(img, (512,512), interpolation='bilinear')
        #label = paddle.vision.transforms.resize(label, (512,512), interpolation='bilinear')
        # 随机选择512的patch
        ps = 256
        hh, ww = label.shape[0], label.shape[1]
        rr     = random.randint(0, hh-ps)
        cc     = random.randint(0, ww-ps)
        img = img[rr:rr+ps, cc:cc+ps,:]
        label = label[rr:rr+ps, cc:cc+ps,:]

        img = img.transpose((2,0,1))
        label = label.transpose((2,0,1))
        
        img = img/255
        label = label/255

        img = paddle.to_tensor(img).astype('float32')
        label = paddle.to_tensor(label).astype('float32')

        return img,label

    def __len__(self):
        return len(self.train_list)

# 对dataloader进行测试

train_dataset=MyDateset()

train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    drop_last=False)

for step, data in enumerate(train_dataloader):
    img, label = data
    print(step, img.shape, label.shape)
    break

W0906 23:22:49.488724  2870 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W0906 23:22:49.492609  2870 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.


0 [16, 3, 256, 256] [16, 3, 256, 256]


**2.定义网络结构**

In [2]:

"""
## Learning Enriched Features for Real Image Restoration and Enhancement
## Syed Waqas Zamir, Aditya Arora, Salman Khan, Munawar Hayat, Fahad Shahbaz Khan, Ming-Hsuan Yang, and Ling Shao
## ECCV 2020
## https://arxiv.org/abs/2003.06792
"""


# --- Imports --- #
import paddle
import paddle.nn as nn
import numpy as np
from paddle.nn import functional as F

##########################################################################
##---------- Selective Kernel Feature Fusion (SKFF) ----------
class SKFF(nn.Layer):
    def __init__(self, in_channels, height=3,reduction=8,bias=False):
        super(SKFF, self).__init__()

        self.height = height
        d = max(int(in_channels/reduction),4)
        
        self.avg_pool = nn.AdaptiveAvgPool2D(1)
        self.conv_du = nn.Sequential(nn.Conv2D(in_channels, d, 1, padding=0, bias_attr=bias), nn.LeakyReLU(0.2))

        self.fcs = nn.LayerList([])
        for i in range(self.height):
            self.fcs.append(nn.Conv2D(d, in_channels, kernel_size=1, stride=1,bias_attr=bias))
        
        self.softmax = nn.Softmax(axis=1)

    def forward(self, inp_feats):
        batch_size = inp_feats[0].shape[0]
        n_feats =  inp_feats[0].shape[1]
        

        inp_feats = paddle.concat(inp_feats, axis=1)
        inp_feats = inp_feats.reshape([batch_size, self.height, n_feats, inp_feats.shape[2], inp_feats.shape[3]])
        
        feats_U = paddle.sum(inp_feats, axis=1)
        feats_S = self.avg_pool(feats_U)
        feats_Z = self.conv_du(feats_S)

        attention_vectors = [fc(feats_Z) for fc in self.fcs]
        attention_vectors = paddle.concat(attention_vectors, axis=1)
        attention_vectors = attention_vectors.reshape([batch_size, self.height, n_feats, 1, 1])
        # stx()
        attention_vectors = self.softmax(attention_vectors)
        
        feats_V = paddle.sum(inp_feats*attention_vectors, axis=1)
        
        return feats_V

class ContextBlock(nn.Layer):

    def __init__(self, n_feat, bias=False):
        super(ContextBlock, self).__init__()

        self.conv_mask = nn.Conv2D(n_feat, 1, kernel_size=1, bias_attr=bias)
        self.softmax = nn.Softmax(axis=2)

        self.channel_add_conv = nn.Sequential(
            nn.Conv2D(n_feat, n_feat, kernel_size=1, bias_attr=bias),
            nn.LeakyReLU(0.2),
            nn.Conv2D(n_feat, n_feat, kernel_size=1, bias_attr=bias)
        )

    def modeling(self, x):
        batch, channel, height, width = x.shape
        input_x = x
        # [N, C, H * W]
        input_x = input_x.reshape([batch, channel, height * width])
        # [N, 1, C, H * W]
        input_x = input_x.unsqueeze(1)
        # [N, 1, H, W]
        context_mask = self.conv_mask(x)
        # [N, 1, H * W]
        context_mask = context_mask.reshape([batch, 1, height * width])
        # [N, 1, H * W]
        context_mask = self.softmax(context_mask)
        # [N, 1, H * W, 1]
        context_mask = context_mask.unsqueeze(3)
        # [N, 1, C, 1]
        context = paddle.matmul(input_x, context_mask)
        # [N, C, 1, 1]
        context = context.reshape([batch, channel, 1, 1])

        return context

    def forward(self, x):
        # [N, C, 1, 1]
        context = self.modeling(x)

        # [N, C, 1, 1]
        channel_add_term = self.channel_add_conv(context)
        x = x + channel_add_term

        return x


##########################################################################
##---------- Spatial Attention ----------
class RCB(nn.Layer):
    def __init__(self, n_feat, kernel_size=3, reduction=8, bias=False, groups=1):
        super(RCB, self).__init__()

        act = nn.LeakyReLU(0.2)

        self.body = nn.Sequential(
            nn.Conv2D(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias_attr=bias, groups=groups),
            act,
            nn.Conv2D(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias_attr=bias, groups=groups)
        )

        self.act = act

        self.gcnet = ContextBlock(n_feat, bias=bias)

    def forward(self, x):
        res = self.body(x)
        res = self.act(self.gcnet(res))
        res += x
        return res


##########################################################################
##---------- Resizing Layers ----------    
class Down(nn.Layer):
    def __init__(self, in_channels, chan_factor, bias=False):
        super(Down, self).__init__()

        self.bot = nn.Sequential(
            nn.AvgPool2D(2, ceil_mode=True, exclusive=False),
            nn.Conv2D(in_channels, int(in_channels * chan_factor), 1, stride=1, padding=0, bias_attr=bias)
        )

    def forward(self, x):
        return self.bot(x)


class DownSample(nn.Layer):
    def __init__(self, in_channels, scale_factor, chan_factor=2, kernel_size=3):
        super(DownSample, self).__init__()
        self.scale_factor = int(np.log2(scale_factor))

        Layers_body = []
        for i in range(self.scale_factor):
            Layers_body.append(Down(in_channels, chan_factor))
            in_channels = int(in_channels * chan_factor)

        self.body = nn.Sequential(*Layers_body)

    def forward(self, x):
        x = self.body(x)
        return x


class Up(nn.Layer):
    def __init__(self, in_channels, chan_factor, bias=False):
        super(Up, self).__init__()

        self.bot = nn.Sequential(
            nn.Conv2D(in_channels, int(in_channels // chan_factor), 1, stride=1, padding=0, bias_attr=bias),
            nn.Upsample(scale_factor=2, mode='bilinear', align_corners=bias)
        )

    def forward(self, x):
        return self.bot(x)


class UpSample(nn.Layer):
    def __init__(self, in_channels, scale_factor, chan_factor=2, kernel_size=3):
        super(UpSample, self).__init__()
        self.scale_factor = int(np.log2(scale_factor))

        Layers_body = []
        for i in range(self.scale_factor):
            Layers_body.append(Up(in_channels, chan_factor))
            in_channels = int(in_channels // chan_factor)

        self.body = nn.Sequential(*Layers_body)

    def forward(self, x):
        x = self.body(x)
        return x


##########################################################################
class GRU_sample(nn.Layer):
    def __init__(self):
        super(GRU_sample, self).__init__()
    def forward(self,x,h_t_1):
        C = x.shape[1]
        z_t = F.sigmoid(x + h_t_1)
        h_hat_t = F.tanh(x + paddle.matmul(z_t, h_t_1))
        h_t = paddle.matmul((1 - z_t), h_t_1) + paddle.matmul(z_t, h_hat_t)
        #conv = nn.Conv2d(in_channels=C, out_channels=C, kernel_size=1, stride=1) 
        conv = nn.Conv2D(C, C, 1, stride=1, padding=0, bias_attr=False)
        y = conv(h_t)
        return y, h_t 

##---------- Multi-Scale Resiudal Block (MRB) ----------
class MRB(nn.Layer):
    def __init__(self, n_feat, height, width, chan_factor, bias, groups):
        super(MRB, self).__init__()

        self.n_feat, self.height, self.width = n_feat, height, width

        self.dau_top = RCB(int(n_feat * chan_factor ** 0), bias=bias, groups=groups)
        self.dau_mid = RCB(int(n_feat * chan_factor ** 1), bias=bias, groups=groups)
        self.dau_bot = RCB(int(n_feat * chan_factor ** 2), bias=bias, groups=groups)

        self.down2 = DownSample(int((chan_factor ** 0) * n_feat), 2, chan_factor)
        self.down4 = nn.Sequential(
            DownSample(int((chan_factor ** 0) * n_feat), 2, chan_factor),
            DownSample(int((chan_factor ** 1) * n_feat), 2, chan_factor)
        )

        self.up21_1 = UpSample(int((chan_factor ** 1) * n_feat), 2, chan_factor)
        self.up21_2 = UpSample(int((chan_factor ** 1) * n_feat), 2, chan_factor)
        self.up32_1 = UpSample(int((chan_factor ** 2) * n_feat), 2, chan_factor)
        self.up32_2 = UpSample(int((chan_factor ** 2) * n_feat), 2, chan_factor)

        self.conv_out = nn.Conv2D(n_feat, n_feat, kernel_size=1, padding=0, bias_attr=bias)

        # only two inputs for SKFF
        self.skff_top = SKFF(int(n_feat * chan_factor ** 0), 2)
        self.skff_mid = SKFF(int(n_feat * chan_factor ** 1), 2)
        self.convGru = GRU_sample()


    def forward(self, x):
        x_top = x.clone()
        x_mid = self.down2(x_top)
        x_bot = self.down4(x_top)

        x_top = self.dau_top(x_top)
        x_mid = self.dau_mid(x_mid)
        x_bot = self.dau_bot(x_bot)

        x_mid = self.skff_mid([x_mid, self.up32_1(x_bot)])
        x_top = self.skff_top([x_top, self.up21_1(x_mid)])

        x_top = self.dau_top(x_top)
        x_mid = self.dau_mid(x_mid)
        x_bot = self.dau_bot(x_bot)
        x_mid = self.skff_mid([x_mid, self.up32_2(x_bot)])
        # x_m,x_up = self.convGru(x_mid, self.up32_2(x_bot)) # add
        # x_mid = self.skff_mid([x_m, x_up])
        x_top = self.skff_top([x_top, self.up21_2(x_mid)])
        

        out = self.conv_out(x_top)
        out = out + x

        return out


##########################################################################
##---------- Recursive Residual Group (RRG) ----------
class RRG(nn.Layer):
    def __init__(self, n_feat, n_MRB, height, width, chan_factor, bias=False, groups=1):
        super(RRG, self).__init__()

        Layers_body = [MRB(n_feat, height, width, chan_factor, bias, groups) for _ in range(n_MRB)]
        Layers_body.append(nn.Conv2D(n_feat, n_feat, kernel_size=3, stride=1, padding=1, bias_attr=bias))
        self.body = nn.Sequential(*Layers_body)

    def forward(self, x):
        res = self.body(x)
        res += x
        return res


##########################################################################
##---------- MIRNet_V2  -----------------------
class MIRNet_v2(nn.Layer):
    def __init__(self,
                 inp_channels=3,
                 out_channels=3,
                 n_feat=80,
                 chan_factor=1.5,
                 n_MRB=2,
                 height=3,
                 width=2,
                 bias=False,
                 task=None
                 ):
        super(MIRNet_v2, self).__init__()
        nn.initializer.set_global_initializer(nn.initializer.KaimingNormal(fan_in=10000.), nn.initializer.Constant(0.0))

        self.task = task
        self.conv_in = nn.Conv2D(inp_channels, n_feat, kernel_size=3, padding=1, bias_attr=bias)

        layers_body = []

        layers_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=1))
        layers_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=2))
        layers_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=4))
        # layers_body.append(RRG(n_feat, n_MRB, height, width, chan_factor, bias, groups=4))

        self.body = nn.Sequential(*layers_body)

        self.conv_out = nn.Conv2D(n_feat, out_channels, kernel_size=3, padding=1, bias_attr=bias)

    def forward(self, inp_img):
        shallow_feats = self.conv_in(inp_img)
        deep_feats = self.body(shallow_feats)

        if self.task == 'defocus_deblurring':
            deep_feats += shallow_feats
            out_img = self.conv_out(deep_feats)

        else:
            out_img = self.conv_out(deep_feats)
            out_img += inp_img

        return out_img
if __name__ == "__main__":
    model = MIRNet_v2(n_feat=64)
    paddle.summary(model, (1, 3, 256, 256))

----------------------------------------------------------------------------------------------------
    Layer (type)                   Input Shape                    Output Shape         Param #    
      Conv2D-1                  [[1, 3, 256, 256]]             [1, 64, 256, 256]        1,728     
    AvgPool2D-1                [[1, 64, 256, 256]]             [1, 64, 128, 128]          0       
     Conv2D-17                 [[1, 64, 128, 128]]             [1, 96, 128, 128]        6,144     
       Down-1                  [[1, 64, 256, 256]]             [1, 96, 128, 128]          0       
    DownSample-1               [[1, 64, 256, 256]]             [1, 96, 128, 128]          0       
    AvgPool2D-2                [[1, 64, 256, 256]]             [1, 64, 128, 128]          0       
     Conv2D-18                 [[1, 64, 128, 128]]             [1, 96, 128, 128]        6,144     
       Down-2                  [[1, 64, 256, 256]]             [1, 96, 128, 128]          0       
    Down

**3.定义loss**

In [3]:
import paddle.nn.functional as F

def gaussian1d(window_size, sigma):
    ###window_size = 11
    x = paddle.arange(window_size,dtype='float32')
    x = x - window_size//2
    gauss = paddle.exp(-x ** 2 / float(2 * sigma ** 2))
    # print('gauss.size():', gauss.size())
    ### torch.Size([11])
    return gauss / gauss.sum()

def create_window(window_size, sigma, channel):
    _1D_window = gaussian1d(window_size, sigma).unsqueeze(1)
    _2D_window = _1D_window.mm(_1D_window.t()).unsqueeze(0).unsqueeze(0)
    # print('2d',_2D_window.shape)
    # print(window_size, sigma, channel)
    return _2D_window.expand([channel,1,window_size,window_size])

def _ssim(img1, img2, window, window_size, channel=3 ,data_range = 255.,size_average=True,C=None):
    # size_average for different channel

    padding = window_size // 2

    mu1 = F.conv2d(img1, window, padding=padding, groups=channel)
    mu2 = F.conv2d(img2, window, padding=padding, groups=channel)
    # print(mu1.shape)
    # print(mu1[0,0])
    # print(mu1.mean())
    mu1_sq = mu1.pow(2)
    mu2_sq = mu2.pow(2)
    mu1_mu2 = mu1 * mu2
    sigma1_sq = F.conv2d(img1 * img1, window, padding=padding, groups=channel) - mu1_sq
    sigma2_sq = F.conv2d(img2 * img2, window, padding=padding, groups=channel) - mu2_sq
    sigma12 = F.conv2d(img1 * img2, window, padding=padding, groups=channel) - mu1_mu2
    if C ==None:
        C1 = (0.01*data_range) ** 2
        C2 = (0.03*data_range) ** 2
    else:
        C1 = (C[0]*data_range) ** 2
        C2 = (C[1]*data_range) ** 2
    # l = (2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1)
    # ssim_map = ((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) * (sigma1_sq + sigma2_sq + C2))
    sc = (2 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2)
    lsc = ((2 * mu1_mu2 + C1) / (mu1_sq + mu2_sq + C1))*sc

    if size_average:
        ### ssim_map.mean()是对这个tensor里面的所有的数值求平均
        return lsc.mean()
    else:
        # ## 返回各个channel的值
        return lsc.flatten(2).mean(-1),sc.flatten(2).mean(-1)

def ms_ssim(
    img1, img2,window, data_range=255, size_average=True, window_size=11, channel=3, sigma=1.5, weights=None, C=(0.01, 0.03)
):

    r""" interface of ms-ssim
    Args:
        img1 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        img2 (torch.Tensor): a batch of images, (N,C,[T,]H,W)
        data_range (float or int, optional): value range of input images. (usually 1.0 or 255)
        size_average (bool, optional): if size_average=True, ssim of all images will be averaged as a scalar
        win_size: (int, optional): the size of gauss kernel
        win_sigma: (float, optional): sigma of normal distribution
        win (torch.Tensor, optional): 1-D gauss kernel. if None, a new kernel will be created according to win_size and win_sigma
        weights (list, optional): weights for different levels
        K (list or tuple, optional): scalar constants (K1, K2). Try a larger K2 constant (e.g. 0.4) if you get a negative or NaN results.
    Returns:
        torch.Tensor: ms-ssim results
    """
    if not img1.shape == img2.shape:
        raise ValueError("Input images should have the same dimensions.")

    # for d in range(len(img1.shape) - 1, 1, -1):
    #     img1 = img1.squeeze(dim=d)
    #     img2 = img2.squeeze(dim=d)

    if not img1.dtype == img2.dtype:
        raise ValueError("Input images should have the same dtype.")

    if len(img1.shape) == 4:
        avg_pool = F.avg_pool2d
    elif len(img1.shape) == 5:
        avg_pool = F.avg_pool3d
    else:
        raise ValueError(f"Input images should be 4-d or 5-d tensors, but got {img1.shape}")

    smaller_side = min(img1.shape[-2:])

    assert smaller_side > (window_size - 1) * (2 ** 4), "Image size should be larger than %d due to the 4 downsamplings " \
                                                        "with window_size %d in ms-ssim" % ((window_size - 1) * (2 ** 4),window_size)

    if weights is None:
        weights = [0.0448, 0.2856, 0.3001, 0.2363, 0.1333]
    weights = paddle.to_tensor(weights)

    if window is None:
        window = create_window(window_size, sigma, channel)
    assert window.shape == [channel, 1, window_size, window_size], " window.shape error"

    levels = weights.shape[0] # 5
    mcs = []
    for i in range(levels):
        ssim_per_channel, cs =  _ssim(img1, img2, window=window, window_size=window_size,
                                       channel=3, data_range=data_range,C=C, size_average=False)
        if i < levels - 1:
            mcs.append(F.relu(cs))
            padding = [s % 2 for s in img1.shape[2:]]
            img1 = avg_pool(img1, kernel_size=2, padding=padding)
            img2 = avg_pool(img2, kernel_size=2, padding=padding)

    ssim_per_channel = F.relu(ssim_per_channel)  # (batch, channel)
    mcs_and_ssim = paddle.stack(mcs + [ssim_per_channel], axis=0)  # (level, batch, channel) 按照等级堆叠
    ms_ssim_val = paddle.prod(mcs_and_ssim ** weights.reshape([-1, 1, 1]), axis=0) # level 相乘
    print(ms_ssim_val.shape)
    if size_average:
        return ms_ssim_val.mean()
    else:
        # 返回各个channel的值
        return ms_ssim_val.flatten(2).mean(1)


class SSIMLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self, window_size=11, channel=3, data_range=255., sigma=1.5):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(SSIMLoss, self).__init__()
       self.data_range = data_range
       self.C = [0.01, 0.03]
       self.window_size = window_size
       self.channel = channel
       self.sigma = sigma
       self.window = create_window(self.window_size, self.sigma, self.channel)
       # print(self.window_size,self.window.shape)
   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用Paddle中相关API自定义的计算逻辑
       # output = xxxxx
       # return output
       return 1-_ssim(input, label,data_range = self.data_range,
                      window = self.window, window_size=self.window_size, channel=3,
                      size_average=True,C=self.C)

class MS_SSIMLoss(paddle.nn.Layer):
   """
   1. 继承paddle.nn.Layer
   """
   def __init__(self,data_range=255., channel=3, window_size=11, sigma=1.5):
       """
       2. 构造函数根据自己的实际算法需求和使用需求进行参数定义即可
       """
       super(MS_SSIMLoss, self).__init__()
       self.data_range = data_range
       self.C = [0.01, 0.03]
       self.window_size = window_size
       self.channel = channel
       self.sigma = sigma
       self.window = create_window(self.window_size, self.sigma, self.channel)
       # print(self.window_size,self.window.shape)
   def forward(self, input, label):
       """
       3. 实现forward函数，forward在调用时会传递两个参数：input和label
           - input：单个或批次训练数据经过模型前向计算输出结果
           - label：单个或批次训练数据对应的标签数据
           接口返回值是一个Tensor，根据自定义的逻辑加和或计算均值后的损失
       """
       # 使用Paddle中相关API自定义的计算逻辑
       # output = xxxxx
       # return output
       return 1-ms_ssim(input, label, data_range=self.data_range,
                      window = self.window, window_size=self.window_size, channel=self.channel,
                      size_average=True,  sigma=self.sigma,
                      weights=None, C=self.C)

class PSNRLoss(paddle.nn.Layer):
   def __init__(self):
       super(PSNRLoss, self).__init__()

   def forward(self, input, label):
       return 100 - 20 * paddle.log10( ((input - label)**2).mean(axis = [1,2,3])**-0.5 )


**4.训练**

In [ ]:

model = MIRNet_v2(n_feat=64)
#paddle.summary(model, (1, 3, 600, 600))
#model = UNet()
# param_dict = paddle.load('MIRNetV2.pdparams')
# model.load_dict(param_dict)
random.seed(1234)
np.random.seed(1234)
paddle.seed(1234)
model.train()

train_dataset=MyDateset()

# 需要接续之前的模型重复训练可以取消注释
# param_dict = paddle.load('./MIRnetV2_model_13.pdparams')
# model.load_dict(param_dict)

train_dataloader = paddle.io.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    drop_last=False)

losspsnr = PSNRLoss()
lossfn = SSIMLoss(window_size=3,data_range=1)

max_epoch=101
scheduler = paddle.optimizer.lr.CosineAnnealingDecay(learning_rate=0.0005, T_max=max_epoch,eta_min=1e-6)
opt = paddle.optimizer.Adam(learning_rate=scheduler, parameters=model.parameters(),weight_decay=1e-8)  
now_step=0
loss1_list = []
loss2_list = []
import time
print(len(train_dataloader))
for epoch in range(max_epoch):
    if epoch >= 0:
        loss1_list_tmp = []
        loss2_list_tmp = []
        for step, data in enumerate(train_dataloader):
            now_step+=1
            img, label = data
            pre = model(img)
            loss1 = lossfn(pre,label).mean()
            loss2 = losspsnr(pre,label).mean()
            # Score=0.5∗ 100 PSNR​+0.5∗MSSSIM
            loss = (500*loss1+loss2)/2
            loss1_list_tmp.append((loss1.numpy()[0]))
            loss2_list_tmp.append((loss2.numpy()[0]))
            loss.backward()     
            opt.step() 
            opt.clear_gradients()
            if now_step%1==0:
                print("epoch: {}, batch: {},loss1 is: {},loss2 is: {}, loss is: {} ".format(epoch, step, loss1.mean().numpy()*500,loss2.mean().numpy(), loss.mean().numpy()))
            # if now_step%1000==0:
            #     paddle.save(model.state_dict(), f"NAFnet_model_{epoch}_{now_step}.pdparams")
            #     time.sleep(50)
        loss1_list.append(sum(loss1_list_tmp)/len(loss1_list_tmp))
        loss2_list.append(sum(loss2_list_tmp)/len(loss2_list_tmp))
        with open("losss_MIRV2.txt","w") as f:
            f.write(str(loss1_list)+"\n")
            f.write(str(loss2_list)+"\n")
        paddle.save(model.state_dict(), f"MIRnetV2_model_{epoch}.pdparams")
        print(loss1_list)
from matplotlib import pyplot as plt
print(loss1_list)
print(loss2_list)
plt.plot(loss1_list,label='loss 1')
plt.plot(loss2_list,label='loss 2')
plt.legend(loc='upper right')
plt.savefig("loss.png")

500
epoch: 0, batch: 0,loss1 is: [27.032911],loss2 is: [77.45364], loss is: [52.24328] 
epoch: 0, batch: 1,loss1 is: [36.336334],loss2 is: [79.37695], loss is: [57.856644] 
epoch: 0, batch: 2,loss1 is: [1.7333329],loss2 is: [74.36154], loss is: [38.047436] 
epoch: 0, batch: 3,loss1 is: [42.632698],loss2 is: [73.25308], loss is: [57.94289] 
epoch: 0, batch: 4,loss1 is: [65.10925],loss2 is: [73.62608], loss is: [69.36767] 
epoch: 0, batch: 5,loss1 is: [56.12868],loss2 is: [77.014305], loss is: [66.571495] 
epoch: 0, batch: 6,loss1 is: [125.44846],loss2 is: [84.18031], loss is: [104.81439] 
epoch: 0, batch: 7,loss1 is: [35.99113],loss2 is: [78.93207], loss is: [57.4616] 
epoch: 0, batch: 8,loss1 is: [117.9074],loss2 is: [82.624626], loss is: [100.266014] 
epoch: 0, batch: 9,loss1 is: [25.478722],loss2 is: [71.43649], loss is: [48.457607] 
epoch: 0, batch: 10,loss1 is: [28.5033],loss2 is: [68.9775], loss is: [48.740402] 
epoch: 0, batch: 11,loss1 is: [32.20731],loss2 is: [68.219025], loss 

In [4]:
! python predict.py data/data154549/train_data_01/deblur_testA/blur_image results

W0630 07:37:57.758509  8064 gpu_context.cc:278] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 10.1
W0630 07:37:57.763079  8064 gpu_context.cc:306] device: 0, cuDNN Version: 7.6.
